In [ ]:
from hdbscan import HDBSCAN
import sklearn
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.cluster import SpectralClustering
from matplotlib.cm import get_cmap
%matplotlib inline

# G = nx.Graph()
# G.add_nodes_from(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K'])
# G.add_edges_from([
#     ('A', 'B'), ('A', 'C'), ('A', 'D'), ('A', 'E'), ('B', 'C'), ('B', 'F'), ('C', 'F'), ('D', 'G'), ('D', 'H'),
#     ('E', 'H'), ('F', 'I'), ('G', 'I'), ('G', 'J'), ('H', 'J'), ('I', 'K'), ('J', 'K'),
# ])
G = nx.erdos_renyi_graph(15, .3, seed=42)
pos_fr = nx.fruchterman_reingold_layout(G, seed=42)
nx.draw(G, pos=pos_fr, node_size=800, node_color='red', with_labels=True)
plt.show()

adj_matrix = nx.to_numpy_matrix(G)
node_list = list(G.nodes()) 

print(adj_matrix)
clusters = SpectralClustering(
    affinity='precomputed', assign_labels='discretize', random_state=42, n_clusters=4
).fit_predict(adj_matrix)
# clusters = HDBSCAN(
#     metric='precomputed'
# ).fit_predict(adj_matrix)
print(node_list)
print(clusters)

cmap = get_cmap('tab20').colors
for i in range(len(node_list)):
    print(node_list[i])
    print(cmap[clusters[i]])
    nx.draw_networkx_nodes(
        G, pos=pos_fr, nodelist={node_list[i]}, node_size=800, node_color=cmap[clusters[i]]
    )
nx.draw_networkx_edges(G, pos=pos_fr)
nx.draw_networkx_labels(G, pos=pos_fr)